In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques chargées avec succès")

✅ Bibliothèques chargées avec succès


In [3]:
# Charger les données depuis train.csv
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

# Afficher les informations de base
print(f"Nombre d'enregistrements : {df.shape[0]}")
print(f"Nombre de colonnes : {df.shape[1]}")
print("\nPremières lignes du dataset :")
print(df.head())
print("\nInformations sur les colonnes :")
print(df.info())

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'

In [ ]:
# Sélectionner uniquement les colonnes numériques
numeric_cols = df.select_dtypes(include=['number']).columns
data_numeric = df[numeric_cols].copy()

print(f"\n✅ Nombre de colonnes numériques : {len(numeric_cols)}")
print(f"Colonnes numériques : {list(numeric_cols)}")

# Vérifier les valeurs manquantes
print("\n📊 Valeurs manquantes par colonne :")
missing_values = data_numeric.isnull().sum()
print(missing_values[missing_values > 0])

# Remplacer les valeurs manquantes par la moyenne de chaque colonne
means = data_numeric.mean()
data_numeric.fillna(means, inplace=True)

print("\n✅ Valeurs manquantes remplacées par la moyenne")
print(f"Total de valeurs manquantes restantes : {data_numeric.isnull().sum().sum()}")


✅ Nombre de colonnes numériques : 38
Colonnes numériques : ['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

📊 Valeurs manquantes par colonne :
LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64

✅ Valeurs manquantes remplacées par la moyenne
Total de valeurs manquantes restantes : 0


In [ ]:
# Vérifier que 'SalePrice' est bien dans les colonnes
if 'SalePrice' not in data_numeric.columns:
    raise ValueError("La colonne 'SalePrice' n'est pas présente dans les données numériques")

# Séparer les features (X) et la cible (y)
X = data_numeric.drop('SalePrice', axis=1)
y = data_numeric['SalePrice']

print(f"\n✅ Séparation effectuée")
print(f"Nombre de features (X) : {X.shape[1]}")
print(f"Nombre d'échantillons : {X.shape[0]}")
print(f"\nStatistiques de la cible (SalePrice) :")
print(y.describe())


✅ Séparation effectuée
Nombre de features (X) : 37
Nombre d'échantillons : 1460

Statistiques de la cible (SalePrice) :
count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64


In [ ]:
# Créer des bins pour la stratification avec pd.cut
# On divise le prix en 5 catégories pour assurer une distribution équilibrée
y_bins = pd.cut(y, bins=5, labels=False)

print("\n📊 Distribution des bins de prix :")
print(pd.Series(y_bins).value_counts().sort_index())

# Diviser les données en train et test avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_bins
)

print(f"\n✅ Split effectué avec stratification")
print(f"Taille du train set : {X_train.shape[0]} échantillons ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Taille du test set : {X_test.shape[0]} échantillons ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nDistribution du prix dans train :")
print(f"  Min: ${y_train.min():,.0f}, Max: ${y_train.max():,.0f}, Mean: ${y_train.mean():,.0f}")
print(f"Distribution du prix dans test :")
print(f"  Min: ${y_test.min():,.0f}, Max: ${y_test.max():,.0f}, Mean: ${y_test.mean():,.0f}")


📊 Distribution des bins de prix :
SalePrice
0    871
1    508
2     70
3      7
4      4
Name: count, dtype: int64

✅ Split effectué avec stratification
Taille du train set : 1168 échantillons (80.0%)
Taille du test set : 292 échantillons (20.0%)

Distribution du prix dans train :
  Min: $34,900, Max: $755,000, Mean: $181,225
Distribution du prix dans test :
  Min: $68,400, Max: $625,000, Mean: $179,707


In [ ]:
# Créer et entraîner le modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

print("\n✅ Modèle de régression linéaire entraîné avec succès")
print(f"Nombre de coefficients : {len(model.coef_)}")
print(f"Intercept : ${model.intercept_:,.2f}")


✅ Modèle de régression linéaire entraîné avec succès
Nombre de coefficients : 37
Intercept : $218,422.74


In [ ]:
# Prédictions sur les ensembles train et test
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Calculer les métriques pour le train set
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

# Calculer les métriques pour le test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)

# Afficher les résultats
print("\n" + "="*60)
print("📊 ÉVALUATION DU MODÈLE")
print("="*60)
print("\n🔹 TRAIN SET :")
print(f"  RMSE : ${train_rmse:,.2f}")
print(f"  R² Score : {train_r2:.4f} ({train_r2*100:.2f}%)")

print("\n🔹 TEST SET :")
print(f"  RMSE : ${test_rmse:,.2f}")
print(f"  R² Score : {test_r2:.4f} ({test_r2*100:.2f}%)")

print("\n🔹 ANALYSE :")
if train_r2 > test_r2 + 0.1:
    print("  ⚠️  Possible surapprentissage (overfitting)")
elif train_r2 < 0.5:
    print("  ⚠️  Sous-apprentissage (underfitting) - modèle trop simple")
else:
    print("  ✅ Performance équilibrée")

print("="*60)


📊 ÉVALUATION DU MODÈLE

🔹 TRAIN SET :
  RMSE : $31,209.09
  R² Score : 0.8482 (84.82%)

🔹 TEST SET :
  RMSE : $50,824.08
  R² Score : 0.5592 (55.92%)

🔹 ANALYSE :
  ⚠️  Possible surapprentissage (overfitting)


In [ ]:
# Sauvegarder le modèle en fichier pickle
model_filename = 'model.pkl'

with open(model_filename, 'wb') as file:
    pickle.dump(model, file)

print(f"\n✅ Modèle sauvegardé dans '{model_filename}'")

# Vérifier que le modèle peut être rechargé
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)
    
# Tester le modèle rechargé
test_prediction = loaded_model.predict(X_test[:5])
print(f"\n✅ Modèle rechargé et testé avec succès")
print(f"Exemple de prédictions : {test_prediction[:3]}")


✅ Modèle sauvegardé dans 'model.pkl'

✅ Modèle rechargé et testé avec succès
Exemple de prédictions : [246180.62921724  89596.91257043 335939.64452261]
